Este endpoint solicita devolver el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [1]:
import pandas as pd
import pyarrow.parquet as pq

In [2]:
df_steam_games = pq.read_table('Datos/df_steam_games_limpio_final.parquet').to_pandas()

In [3]:
df_reviews = pq.read_table('Datos/df_reviews_sentimiento.parquet').to_pandas()

Selecciono las columnas de los dos dataframes que me van a ser útiles para este análisis en particular

In [4]:
df_steam_games = df_steam_games[['id','developer','anio']]
df_reviews = df_reviews[['item_id','recommend','sentiment_analysis']]

In [5]:
df = df_steam_games.groupby('id').first().reset_index()

In [6]:
df = df[df['anio']!=0]

In [7]:
df.head()

,id,developer,anio
1,10,valve,2000
2,20,valve,1999
3,30,valve,2003
4,40,valve,2001
5,50,gearbox software,1999


In [8]:
df_reviews.head()

,item_id,recommend,sentiment_analysis
0,1250,true,2
1,22200,true,2
2,43110,true,2
3,251610,true,2
4,227300,true,0


Una vez seleccionadas las columnas útiles, realizo el merge (inner join) para quedarme con un nuevo dataframe llamado "resultado" que contenga sólo los elementos correspondientes a los id que sean match entre los dos dataframes

In [10]:
df_nuevo = df.merge(df_reviews, left_on='id', right_on='item_id', how='inner')

In [11]:
df_nuevo.drop('id', axis=1, inplace=True)

Sólo quiero los valores en que 'recommend' sea True y el sentiment análisis sea 2 (bueno), como pide la consigna

In [12]:
df_nuevo = df_nuevo[df_nuevo['recommend'] == 'true']

In [13]:
df_nuevo = df_nuevo[df_nuevo['sentiment_analysis'] == 2]

In [14]:
df_nuevo[(df_nuevo['anio'] == 2000) & (df_nuevo['developer'] == 'valve')].shape

(29, 5)

In [15]:
cantidad = df_nuevo[(df_nuevo['anio'] == 2000)].groupby('developer').count()

In [16]:
cantidad

,anio,item_id,recommend,sentiment_analysis
developer,,,,
bioware corporation,1,1,1,1
funcom,1,1,1,1
ion storm,8,8,8,8
looking glass studios,1,1,1,1
rebellion,3,3,3,3
shiny entertainment,1,1,1,1
valve,29,29,29,29


In [17]:
cantidad.reset_index(inplace=True)

In [18]:
cantidad = cantidad.sort_values(by='anio', ascending=False)

Me quedo con las columnas 'developer' y 'año'

In [19]:
cantidad = cantidad[['developer','anio']]

In [20]:
cantidad

,developer,anio
6,valve,29
2,ion storm,8
4,rebellion,3
0,bioware corporation,1
1,funcom,1
3,looking glass studios,1
5,shiny entertainment,1


In [21]:
cantidad.set_index('developer', inplace=True)

In [22]:
cantidad.columns = ['Cantidad']

Me quedo con los tres developers con más reviews positivos para el año dado

In [23]:
primeros_tres = cantidad.iloc[:3]

In [24]:
primeros_tres.to_dict()

{'Cantidad': {'valve': 29, 'ion storm': 8, 'rebellion': 3}}